# Warm-up example


## Some key concepts

Before getting into formalities, let us start with a concrete example. Consider the following dataset.

![Plain data](img/outliers_data_plain.png)

This is the well-known ["Belgium Phone Calls 1950-1973"](https://stat.ethz.ch/R-manual/R-devel/library/MASS/html/phones.html) dataset \[Ven02\], normalized such that the inputs (horizontal axis) take values between $0$ and $x_{\max}$, and the outputs (vertical axis) take values between $0$ and $y_{\max}$. The exact maximum values used are not critical to our discussion here, and the real-world context in which this data originally arose will be left abstract in the subsequent paragraphs.

When the task is "regression," i.e., prediction of the outputs based upon the inputs, the simplest and most traditional approach to this problem is *ordinary least squares* (OLS). Using the [SciPy implementation of OLS](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lstsq.html), we arrive at the following regression line.

![OLS solution](img/outliers_plain_ols.png)

How "good" is this OLS solution? From the perspective of finding a <u>linear</u> predictor that minimizes the <u>average</u> of the <u>squared prediction error</u>, then the OLS solution is of course *optimal*. When considering the performance of learning systems, however, it is not enough to consider only the data at hand; we also hope to achieve good prediction on new data, that is, *off-sample*. Let us consider two possible scenarios illustrated in the two plots below.

![OLS solution](img/outliers_two_scenarios.png)

In both of these plots, the data we have access to now (at "training" time) is represented using black points, whereas gray points represent a larger sample from an underlying data-generating process that we do not have access to. We may of course eventually have access to this data, i.e., at "test" time, but the essence of the learning problem is how well we can do at test time using the limited information available at training time. In the left-hand plot, the relationship between the inputs and outputs is essentially linear and slow-growing, with a few rare exceptions. On the other hand, in the right-hand plot, the true relationship appears to be rather non-linear.

With these two scenarios in mind, let us return to the question of how "good" the OLS solution is.

In the essentially linear scenario, we see that at test time, a different choice of regression line (smaller slope) would yield much better predictions on the vast majority of the data points; if the rare exceptions can be ignored, it is natural to be dissatisfied with the OLS solution. That is, the linear *model* is correct, but the output of the learning algorithm that we have called "OLS" is unsatisfactory.

On the other hand, in the non-linear scenario, one expects that the sharp upward trend is *not* a rare exception that can be ignored, but rather a fundamental non-linearity in the relationship between the inputs and outputs. In this case, while our linear model is very limited in its expressive power, given this model, the OLS solution itself is actually quite good.

To summarize, in the former scenario, the model is correct, but the learning algorithm is poor. In the latter scenario, the model is wrong, but the learning algorithm has done a good job. In this article, we are interested in general principles for learning algorithm design (rather than model design), and we will proceed by considering how to modify the OLS algorithm within the confines of the essentially linear scenario.

## Initial strategy: changing the loss function

In considering an alternative to the OLS algorithm, let us recall its traits, as summarized in the following sentence given a few paragraphs back.

> From the perspective of finding a <u>linear</u> predictor that minimizes the <u>average</u> of the <u>squared prediction error</u>, then the OLS solution is of course *optimal*.

We have underlined key terms for a reason; these three points characterize the OLS solution plotted earlier. To make things more precise, let us introduce a bit of formal notation. The first term ("<u>linear</u>") relates to model design, and means that we will use a predictor of the form $h(x) = ax + b$, where $a$ and $b$ are arbitrary real values. Using $\mathcal{H}$ as general-purpose notation for our model (or "hypothesis class"), here we have $\mathcal{H} = \{x \mapsto ax + b: a, b \in \mathbb{R}\}$. As for the other two terms ("<u>average</u>", "<u>squared prediction error</u>"), these relate to algorithm design, and they mean that the learning algorithm returns a solution to the optimization task

\begin{align}
\min_{h \in \mathcal{H}} \, \frac{1}{n}\sum_{i=1}^{n}\ell(h;X_{i},Y_{i})
\end{align}

where the loss function takes the form $\ell(h;x,y)=(h(x)-y)^{2}$ and $\{(X_{i},Y_{i}): i=1,\ldots,n\}$ denotes the data sample available at training time. When $\mathcal{H}$ is a linear model, this optimization problem is very simple; it reduces to solving a system of linear equations. This means that in implementing the OLS procedure, one can leverage very efficient linear algebra software libraries. While this simplicity is appealing, recall that we were *dissatisfied* with the OLS solution in the linear scenario. Let us consider some typical modifications to the learning algorithm.

Since this dataset is small both in size and dimensionality, by plotting the data it is easy to identify the errant data points. If we manually remove the six points that deviate far from the slow-growth trend of the majority and run the OLS procedure on this smaller dataset, we end up with a very different predictor (shown using a dashed line).

![OLS after removal](img/outliers_plain_ols_removed.png)

While manual removal of errant points does work in very simple settings such as this one, larger datasets with more inputs makes the identification of "outliers" a highly non-trivial problem. One of the most common alternative approaches is that of *least absolute deviations* (LAD); this procedure is identical to the OLS procedure described, except that a different loss function is used, namely the absolute error $\ell(h;x,y) = \lvert h(x)-y \rvert$. Using the [LAD implementation in scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.QuantileRegressor.html), one gets the following predictor (drawn using a solid blue line).

![LAD and OLS comparison](img/outliers_lad.png)

Note that the outputs of the algorithms "vanilla LAD" and "OLS after removing outliers" are very similar, but the former gives us such a predictor without having to specify outliers manually. On the other hand, the "vanilla OLS" and "vanilla LAD" outputs are clearly very different.

Based on these observations, one might be inclined to make the following broad conclusion: "LAD is insensitive to outliers." Indeed, for this particular data set, the output of LAD *is* insensitive to the outlying data points. However, is this a property we should expect in general? As highlighted earlier, both OLS and LAD are defined in terms of optimizing the <u>average</u> loss. Since the objective function puts the same weight on every loss value, an insensitivity to outliers is by no means an explicit part of the algorithm design, and in principle, there is no reason to expect it to hold in general.

Indeed, it is easy to construct an example where both OLS and LAD ignore the trend of the majority.

![Example where LAD is drawn astray](img/outliers_modified_lev.png)

We have taken the original dataset and modified just one of the original outlying data points, simply multiplying this one data point $(X_{i},Y_{i})$ by 5. Considering the first-order optimality conditions for LAD under the linear model, we require

\begin{align}
\sum_{i=1}^{n}\text{sign}(h(X_{i})-Y_{i}) \, X_{i} = 0.
\end{align}

This shows how the value of $Y_{i}$ can be arbitrarily large without causing problems, whereas an errant value of $X_{i}$ can be very troublesome. Such data points are said to be "high leverage" in the context of robust linear regression \[RL87\]. We thus see how using LAD instead of OLS does not necessarily realize the desired insensitivity to errant observations, and this should not be surprising, since all OLS and LAD promise is a *small average loss*, nothing more, nothing less.

## A new perspective: seeking a desirable loss distribution

With this context in place, we can get to the central point of this article, namely the notion that it might be fruitful to __design learning algorithms based on properties we desire in the loss distribution__. It goes without saying that in an ideal world, we would like all the losses to take on the smallest possible value, both on- and off-sample. Unfortunately, the reality of machine learning in the wild is that we only have limited data and prior knowledge at training time, and thus _"make all losses minimal!"_ is a totally unrealistic goal. Not all losses can be made minimal. Tradeoffs are inevitable. In order to responsibly design effective and transparent learning systems, we need the _vocabulary_ to make these tradeoffs explicit. By placing our focus on properties of the loss distribution, we have a medium by which we can translate decisions about performance tradeoffs into concrete mathematical notions and executable code.

To motivate this idea further, let us return briefly to the original, unmodified dataset. This is the "nice" setting in which we know that the LAD solution is desirable. We can go one step further and say that the _loss distribution_ incurred by the LAD solution is also desirable. Below we plot a histogram of the (training) losses incurred by the two solutions being compared (red is OLS, blue is LAD). Vertical rules denote the average loss (dashed) and median loss (dotted).

![LAD and OLS comparison (training)](img/outliers_loss_dists.png)

Note that there are some clear differences between the loss distributions of each method. On one hand, the majority of the losses incurred by LAD are smaller than those of OLS, but on the other hand, the LAD solution incurs more extremely large losses. This results in a large gap between the median and mean for LAD. These trends become more visually salient with a larger sample; let us next look at the off-sample losses, corresponding to the gray points plotted previously in the linear scenario.

![LAD and OLS comparison (test)](img/outliers_loss_dists_te.png)

Since the outlying losses are difficult to see, we also give a zoomed-in view of these histograms.

![LAD and OLS comparison (test, zoomed, square)](img/outliers_loss_dists_te_zoom_sq.png)
![LAD and OLS comparison (test, zoomed, abs)](img/outliers_loss_dists_te_zoom_abs.png)

Again we see that regardless of the loss function used to measure prediction error, the majority of the losses are small for LAD, but at the cost of more extremely large errors than OLS. Put very roughly, we'd like a distribution with a very small median, and we are willing to tolerate some heavy tails on the upside (and thus a larger mean). Why focus on these properties? The reason is that these properties transcend this particular dataset. We want a loss distribution with these properties in the original "nice" setting, in the modified setting (with a high-leverage point) where LAD falls apart, and indeed all settings in-between where the majority of data points are representative of the true underlying (here, linear) relationship between inputs and outputs.

The key remaining question: how can we build a learning algorithm that achieves such a distribution? More generally, as our definition of "desirable distribution" changes, what algorithmic modifications are required? Providing some preliminary answers to these questions is the main goal of this article, and we will take it at a leisurely pace, step-by-step.

## A concrete alternative

As a first step, and to wrap up this first section, we will consider the most direct approach to the current regression task. Recall the desired properties mentioned earlier:

> Put very roughly, we'd like a distribution with a very small median, and we are willing to tolerate some heavy tails on the upside (and thus a larger mean).

If we want to ensure the median is small, then the most direct method is to use the median loss as an objective function to be passed to an optimization procedure. To make this explicit, let us denote each sample loss by $\mathrm{L}_{i}(h) = \ell(h;X_{i},Y_{i})$, and use the notation $\mathrm{L}_{(i)}$ to denote the $i$th largest value, i.e., we have

\begin{align}
\mathrm{L}_{(1)}(h) \leq \mathrm{L}_{(2)}(h) \leq \cdots \leq \mathrm{L}_{(n)}(h).
\end{align}

Note that the ordering of points may naturally change with the choice of $h$. With this notation in place, the program to solve would then be

\begin{align}
\min_{h \in \mathcal{H}} \, \mathrm{L}_{(k)}(h), \, \text{ where } k = \lceil n/2 \rceil.
\end{align}

Statistically speaking, this is a perfectly natural idea, which is of interest regardless of the nature of the model $\mathcal{H}$ or the form of the loss function $\ell(h;x,y)$. On the other hand, from a practical perspective, this idea is extremely naive since actually implementing this minimization is much more difficult than just minimizing the average; even if the loss function is smooth and convex in $h$, these properties need not hold for the median. In the special case where $\mathcal{H}$ is a linear model and $\ell(h;x,y)$ is the squared error, this method was studied almost 40 years ago under the name _least median of squares_ (LMedS), with core ideas dating back much further \[Rou84,RL87\]. A practical implementation of LMedS is available in [the MASS package for R](https://stat.ethz.ch/R-manual/R-patched/library/MASS/html/lqs.html), but in machine learning we typically want flexibility to deal with a wide range of models and loss functions, so for some perspective, let us see how an off-the-shelf solver deals with this task. We use the `minimize` function [built in to SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) using the Nelder-Mead method, initialized by setting $h(x) = ax + b$ with $a$ and $b$ independently and uniformly sampled from the interval $[-5,5]$. We plot the LMedS solution obtained on the original dataset below.

![LMS on original data](img/outliers_lml.png)

It is evident that the LMedS solution (green) is very similar to the LAD solution (blue) and the solution obtained using manual outlier removal (red, dashed). Both of these alternative solutions achieve a loss distribution with a small median, so this is a natural outcome. The critical remaining question is how well this strategy works on the modified dataset, where the LAD approach is known to fail. We plot the resulting LMS solution below.

![LMS on modified data](img/outliers_modified_lev_lml.png)

Fortunately, we see that in this setting as well, the LMedS solution is as we would hope. It is also of some interest that the output is almost identical if we replace the squared error in the LMedS objective by the absolute error. This procedure does not scale well as the number of parameters grows, and is only meant as a lucid example to illustrate algorithm design principles we are interested in. As a key take-away from this example, we have seen that by considering the properties that we desire in the loss distribution, we were able to design a simple and transparent learning algorithm that outperforms OLS and LAD, without having to manually remove the outliers or re-design the loss function.

In general, of course, our definition of "desirable loss distribution" may change significantly depending on the problem of interest. In the sections to follow, we will generalize our formulation, and consider a wide range of (quantifiable) properties of the underlying loss distribution.

___

## Additional notes

### Loss distribution properties and robust statistics

It must be recognized that the basic idea of looking at properties of the loss distribution besides the average is well-engrained into the methodology of robust statistics. Here are a few useful references.

- Handy list of robust linear regression methods \[[URL](https://arxiv.org/pdf/1404.6274.pdf)\].
- A useful textbook reference on computational procedures for robust regression \[RL87\].
- Original LMedS paper is also an excellent reference \[Rou84\]. 


### Different loss function, different solution

Changing the loss function has an interesting effect on the kinds of predictors that are optimal. Let $(X,Y)$ be an as-yet unobserved random "test" point.  Consider two predictors: $h_{\text{ave}}(x) = \mathbf{E}[Y \vert X=x]$ and $h_{\text{med}}(x) = \mathrm{med}[Y \vert X=x]$, namely the expected value and median of $Y$, conditioned on $X$. If the conditional distribution of $Y$ is symmetrical, then these predictors are identical (since the mean and median are equal), but otherwise, they can be very different. That said, both of these predictors can be considered "optimal," depending on the precise method by which we measure the prediction error \[Kol97\]. More precisely, for any choice of $x$ and $h$, it can be proved that

\begin{align}
\mathbf{E}[(h_{\text{ave}}(x)-Y)^{2} \, \vert \, X=x] \leq \mathbf{E}[(h(x)-Y)^{2} \, \vert \, X=x]
\end{align}

and

\begin{align}
\mathbf{E}[\lvert h_{\text{med}}(x)-Y \rvert \, \vert \, X=x] \leq \mathbf{E}[\lvert h(x)-Y \rvert \, \vert \, X=x].
\end{align}

If we want good average prediction in terms of the squared error, then the (conditional) average is best, but if we want good average prediction in terms of the absolute error, then the (conditional) median is best. Taking expectation over $X$, and denoting the joint distribution by $(X,Y) \sim \mu$, it follows that

\begin{align}
h_{\text{ave}} \in \arg\min_{h} \, \mathbf{E}_{\mu} (h(X)-Y)^{2}, \quad h_{\text{med}} \in \arg\min_{h} \, \mathbf{E}_{\mu} \lvert h(X)-Y \rvert.
\end{align}

If the sampled points $(X_{1},Y_{1}),\ldots,(X_{n},Y_{n})$ follow the same distribution as $(X,Y)$, then the OLS and LAD procedures discussed can be considered empirical approximations to $h_{\text{ave}}$ and $h_{\text{med}}$ respectively. By changing the loss function from squared error to absolute error, we are essentially changing the ultimate goal of the learning problem. As the goal changes, it should not be surprising when qualitatively distinct solutions arise. Broadly speaking, of course, the goal of "good prediction of $Y$ based on $X$" is still the same, but this example shows how the precise definition of "good" can have a big impact on the output of general purpose learning algorithms.

### Achieving a similar effect with loss function design

Of course, in principle it is possible to design a non-convex loss function $\ell(h;x,y)$ such that losses beyond a certain threshold are not given any additional weight, but setting this threshold requires a certain amount of extra information about the appropriate scale the losses should take.

## References

- \[Ven02\] Venables, W. N. and Ripley, B. D. (2002) Modern Applied Statistics with S. Fourth edition. Springer.
- \[Kol97\] Koltchinskii, V. I. (1997). M-estimation, convexity and quantiles. The Annals of Statistics, pages 435-477.
- \[Rou84\] P.J. Rousseeuw. (1984) Least Median of Squares Regression. Journal of the American Statistical Association, 79(388):871-880. Taylor & Francis.
- \[RL87\] P.J. Rousseeuw and A.M. Leroy. Robust Regression and Outlier Detection. John Wiley & Sons, New York, 1987.


___